# Imports

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing

# Load Dataset

In [ ]:
original_wavedata=pd.read_csv('data files/townsville-wavedata-1975-2019.csv', low_memory=False)
pd.set_option('display.float_format', lambda x:'%f'%x)

In [ ]:
original_wavedata['Date/Time']=pd.to_datetime(original_wavedata['Date/Time'])
original_wavedata=original_wavedata.set_index(pd.DatetimeIndex(original_wavedata['Date/Time']))

# Function Definitions

In [ ]:
def normalise_dataframe(dataframe):
    x=dataframe
    min_max_scaler=preprocessing.MinMaxScaler()
    x_scaled=min_max_scaler.fit_transform(x)
    normalised_dataframe=pd.DataFrame(x_scaled, columns=dataframe.columns, index=dataframe.index)
    return normalised_dataframe

# Variable Declarations

## Direction True North

In [ ]:
original_dir_tp_months={}
for i in range(1, 13):
    original_dir_tp_months[i]=original_wavedata[original_wavedata["Date/Time"].dt.month==i].drop(["Date/Time", "Hs", "Hmax", "Tz", "Tp", "SST"], axis=1)

In [ ]:
original_dir_tp_summer=original_dir_tp_months[1].append(original_dir_tp_months[2].append(original_dir_tp_months[12]))
original_dir_tp_autumn=original_dir_tp_months[3].append(original_dir_tp_months[4].append(original_dir_tp_months[5]))
original_dir_tp_winter=original_dir_tp_months[6].append(original_dir_tp_months[7].append(original_dir_tp_months[8]))
original_dir_tp_spring=original_dir_tp_months[9].append(original_dir_tp_months[10].append(original_dir_tp_months[11]))

In [ ]:
original_dir_tp_weeks={}
for i in range(1, 53):
    original_dir_tp_weeks[i]=original_wavedata[original_wavedata["Date/Time"].dt.week==i].drop(["Date/Time", "Hs", "Hmax", "Tz", "Tp", "SST"], axis=1)

## Sea Surface Temperature

In [ ]:
original_sst_months={}
for i in range(1, 13):
    original_sst_months[i]=original_wavedata[original_wavedata["Date/Time"].dt.month==i].drop(["Date/Time", "Hs", "Hmax", "Tz", "Tp", "Dir_Tp TRUE"], axis=1)

In [ ]:
original_sst_summer=original_sst_months[1].append(original_sst_months[2].append(original_sst_months[12]))
original_sst_autumn=original_sst_months[3].append(original_sst_months[4].append(original_sst_months[5]))
original_sst_winter=original_sst_months[6].append(original_sst_months[7].append(original_sst_months[8]))
original_sst_spring=original_sst_months[9].append(original_sst_months[10].append(original_sst_months[11]))

In [ ]:
original_sst_weeks={}
for i in range(1, 53):
    original_sst_weeks[i]=original_wavedata[original_wavedata["Date/Time"].dt.week==i].drop(["Date/Time", "Hs", "Hmax", "Tz", "Tp", "Dir_Tp TRUE"], axis=1)

### Day

In [ ]:
original_sst_months_day={}
for i in range(1, 13):
    original_sst_months_day[i]=original_sst_months[i].between_time('6:00', '18:00')

In [ ]:
original_sst_summer_day=original_sst_months_day[1].append(original_sst_months_day[2].append(original_sst_months_day[12]))
original_sst_autumn_day=original_sst_months_day[3].append(original_sst_months_day[4].append(original_sst_months_day[5]))
original_sst_winter_day=original_sst_months_day[6].append(original_sst_months_day[7].append(original_sst_months_day[8]))
original_sst_spring_day=original_sst_months_day[9].append(original_sst_months_day[10].append(original_sst_months_day[11]))

In [ ]:
original_sst_weeks_day={}
for i in range(1, 53):
    original_sst_weeks_day[i]=original_wavedata[original_wavedata["Date/Time"].dt.week==i].drop(["Date/Time", "Hs", "Hmax", "Tz", "Tp", "Dir_Tp TRUE"], axis=1).between_time('6:00', '18:00')

### Night

In [ ]:
original_sst_months_night={}
for i in range(1, 13):
    original_sst_months_night[i]=original_sst_months[i].between_time('18:00', '6:00')

In [ ]:
original_sst_summer_night=original_sst_months_night[1].append(original_sst_months_night[2].append(original_sst_months_night[12]))
original_sst_autumn_night=original_sst_months_night[3].append(original_sst_months_night[4].append(original_sst_months_night[5]))
original_sst_winter_night=original_sst_months_night[6].append(original_sst_months_night[7].append(original_sst_months_night[8]))
original_sst_spring_night=original_sst_months_night[9].append(original_sst_months_night[10].append(original_sst_months_night[11]))

In [ ]:
original_sst_weeks_night={}
for i in range(1, 53):
    original_sst_weeks_night[i]=original_wavedata[original_wavedata["Date/Time"].dt.week==i].drop(["Date/Time", "Hs", "Hmax", "Tz", "Tp", "Dir_Tp TRUE"], axis=1).between_time('18:00', '6:00')

# Remove Outliers and Fill NULL Values

## Direction True North

### Month

#### Remove Outliers

In [ ]:
dir_tp_months_removed_outliers={}
for i in range(1, 13):
    dir_tp_months_removed_outliers[i]=original_dir_tp_months[i][original_dir_tp_months[i].apply(lambda x: np.abs(x - x.mean()) / x.std() < 3).all(axis=1)]

#### Calculate Means

In [ ]:
dir_tp_months_mean={}
for i in range(1, 13):
    dir_tp_months_mean[i]=dir_tp_months_removed_outliers[i].mean()

#### Create Adjusted Dataframe

In [ ]:
dir_tp_months_adjusted=original_wavedata.drop(["Hs", "Hmax", "Tz", "Tp", "SST"], axis=1)
for i in range(1, 13):
    dir_tp_months_adjusted[dir_tp_months_adjusted["Date/Time"].dt.month==i]=dir_tp_months_adjusted[dir_tp_months_adjusted["Date/Time"].dt.month==i].replace(np.nan, dir_tp_months_mean[i])

#### Remove Outliers from Adjusted Dataframe

In [ ]:
dir_tp_months_adjusted=dir_tp_months_adjusted.drop(["Date/Time"], axis=1)
dir_tp_months_adjusted=dir_tp_months_adjusted[dir_tp_months_adjusted.apply(lambda x: np.abs(x - x.mean()) / x.std() < 3).all(axis=1)]

### Seasonal

#### Remove Outliers

In [ ]:
dir_tp_summer_removed_outliers=original_dir_tp_summer[original_dir_tp_summer.apply(lambda x: np.abs(x - x.mean()) / x.std() < 3).all(axis=1)]
dir_tp_autumn_removed_outliers=original_dir_tp_autumn[original_dir_tp_autumn.apply(lambda x: np.abs(x - x.mean()) / x.std() < 3).all(axis=1)]
dir_tp_winter_removed_outliers=original_dir_tp_winter[original_dir_tp_winter.apply(lambda x: np.abs(x - x.mean()) / x.std() < 3).all(axis=1)]
dir_tp_spring_removed_outliers=original_dir_tp_spring[original_dir_tp_spring.apply(lambda x: np.abs(x - x.mean()) / x.std() < 3).all(axis=1)]

#### Calculate Mean

In [ ]:
dir_tp_summer_mean=dir_tp_summer_removed_outliers.mean()
dir_tp_autumn_mean=dir_tp_autumn_removed_outliers.mean()
dir_tp_winter_mean=dir_tp_winter_removed_outliers.mean()
dir_tp_spring_mean=dir_tp_spring_removed_outliers.mean()

#### Create Adjusted Dataframe

In [ ]:
dir_tp_summer_adjusted=original_dir_tp_summer
dir_tp_autumn_adjusted=original_dir_tp_autumn
dir_tp_winter_adjusted=original_dir_tp_winter
dir_tp_spring_adjusted=original_dir_tp_spring
dir_tp_summer_adjusted=dir_tp_summer_adjusted.replace(np.nan, dir_tp_summer_mean)
dir_tp_autumn_adjusted=dir_tp_autumn_adjusted.replace(np.nan, dir_tp_autumn_mean)
dir_tp_winter_adjusted=dir_tp_winter_adjusted.replace(np.nan, dir_tp_winter_mean)
dir_tp_spring_adjusted=dir_tp_spring_adjusted.replace(np.nan, dir_tp_spring_mean)

#### Remove Outliers from Adjusted Dataframe

In [ ]:
dir_tp_summer_adjusted=dir_tp_summer_adjusted[dir_tp_summer_adjusted.apply(lambda x: np.abs(x - x.mean()) / x.std() < 3).all(axis=1)]
dir_tp_autumn_adjusted=dir_tp_autumn_adjusted[dir_tp_autumn_adjusted.apply(lambda x: np.abs(x - x.mean()) / x.std() < 3).all(axis=1)]
dir_tp_winter_adjusted=dir_tp_winter_adjusted[dir_tp_winter_adjusted.apply(lambda x: np.abs(x - x.mean()) / x.std() < 3).all(axis=1)]
dir_tp_spring_adjusted=dir_tp_spring_adjusted[dir_tp_spring_adjusted.apply(lambda x: np.abs(x - x.mean()) / x.std() < 3).all(axis=1)]
dir_tp_season_adjusted=dir_tp_summer_adjusted.append(dir_tp_autumn_adjusted.append(dir_tp_winter_adjusted.append(dir_tp_spring_adjusted)))

### Weeks

#### Remove Outliers

In [ ]:
dir_tp_weeks_removed_outliers={}
for i in range(1, 53):
    dir_tp_weeks_removed_outliers[i]=original_dir_tp_weeks[i][original_dir_tp_weeks[i].apply(lambda x: np.abs(x - x.mean()) / x.std() < 3).all(axis=1)]

#### Calculate Means

In [ ]:
dir_tp_weeks_mean={}
for i in range(1, 53):
    dir_tp_weeks_mean[i]=dir_tp_weeks_removed_outliers[i].mean()

#### Create Adjusted Dataframe

In [ ]:
dir_tp_weeks_adjusted=original_wavedata.drop(["Hs", "Hmax", "Tz", "Tp", "SST"], axis=1)
for i in range(1, 53):
    dir_tp_weeks_adjusted[dir_tp_weeks_adjusted["Date/Time"].dt.week==i]=dir_tp_weeks_adjusted[dir_tp_weeks_adjusted["Date/Time"].dt.week==i].replace(np.nan, dir_tp_weeks_mean[i])

#### Remove Outliers from Adjusted Dataframe

In [ ]:
dir_tp_weeks_adjusted=dir_tp_weeks_adjusted.drop(["Date/Time"], axis=1)
dir_tp_weeks_adjusted=dir_tp_weeks_adjusted[dir_tp_weeks_adjusted.apply(lambda x: np.abs(x - x.mean()) / x.std() < 3).all(axis=1)]

## Sea Surface Temperature

### Day

#### Months

##### Remove Outliers

In [ ]:
sst_months_day_removed_outliers={}
for i in range(1, 13):
    sst_months_day_removed_outliers[i]=original_sst_months_day[i][original_sst_months_day[i].apply(lambda x: np.abs(x - x.mean()) / x.std() < 3).all(axis=1)]

##### Calculate Means

In [ ]:
sst_months_day_mean={}
for i in range(1, 13):
    sst_months_day_mean[i]=sst_months_day_removed_outliers[i].mean()

##### Create Adjusted Dataframes

In [ ]:
sst_months_day_adjusted=original_wavedata.drop(["Hs", "Hmax", "Tz", "Tp", "Dir_Tp TRUE"], axis=1).between_time('6:00', '18:00')
for i in range(1, 13):
    sst_months_day_adjusted[sst_months_day_adjusted["Date/Time"].dt.month==i]=sst_months_day_adjusted[sst_months_day_adjusted["Date/Time"].dt.month==i].replace(np.nan, sst_months_day_mean[i])

##### Remove Outliers from Adjusted Dataframes

In [ ]:
sst_months_day_adjusted=sst_months_day_adjusted.drop(["Date/Time"], axis=1)
sst_months_day_adjusted=sst_months_day_adjusted[sst_months_day_adjusted.apply(lambda x: np.abs(x - x.mean()) / x.std() < 3).all(axis=1)]

#### Seasons

##### Remove Outliers

In [ ]:
sst_summer_day_removed_outliers=original_sst_summer_day[original_sst_summer_day.apply(lambda x: np.abs(x - x.mean()) / x.std() < 3).all(axis=1)]
sst_autumn_day_removed_outliers=original_sst_autumn_day[original_sst_autumn_day.apply(lambda x: np.abs(x - x.mean()) / x.std() < 3).all(axis=1)]
sst_winter_day_removed_outliers=original_sst_winter_day[original_sst_winter_day.apply(lambda x: np.abs(x - x.mean()) / x.std() < 3).all(axis=1)]
sst_spring_day_removed_outliers=original_sst_spring_day[original_sst_spring_day.apply(lambda x: np.abs(x - x.mean()) / x.std() < 3).all(axis=1)]

##### Calculate Means

In [ ]:
sst_summer_day_mean=sst_summer_day_removed_outliers.mean()
sst_autumn_day_mean=sst_autumn_day_removed_outliers.mean()
sst_winter_day_mean=sst_winter_day_removed_outliers.mean()
sst_spring_day_mean=sst_spring_day_removed_outliers.mean()

##### Create Adjusted Dataframes

In [ ]:
sst_summer_day_adjusted=original_sst_summer_day
sst_autumn_day_adjusted=original_sst_autumn_day
sst_winter_day_adjusted=original_sst_winter_day
sst_spring_day_adjusted=original_sst_spring_day
sst_summer_day_adjusted=sst_summer_day_adjusted.replace(np.nan, sst_summer_day_mean)
sst_autumn_day_adjusted=sst_autumn_day_adjusted.replace(np.nan, sst_autumn_day_mean)
sst_winter_day_adjusted=sst_winter_day_adjusted.replace(np.nan, sst_winter_day_mean)
sst_spring_day_adjusted=sst_spring_day_adjusted.replace(np.nan, sst_spring_day_mean)

##### Remove Outliers from Adjusted Dataframes

In [ ]:
sst_summer_day_adjusted=sst_summer_day_adjusted[sst_summer_day_adjusted.apply(lambda x: np.abs(x - x.mean()) / x.std() < 3).all(axis=1)]
sst_autumn_day_adjusted=sst_autumn_day_adjusted[sst_autumn_day_adjusted.apply(lambda x: np.abs(x - x.mean()) / x.std() < 3).all(axis=1)]
sst_winter_day_adjusted=sst_winter_day_adjusted[sst_winter_day_adjusted.apply(lambda x: np.abs(x - x.mean()) / x.std() < 3).all(axis=1)]
sst_spring_day_adjusted=sst_spring_day_adjusted[sst_spring_day_adjusted.apply(lambda x: np.abs(x - x.mean()) / x.std() < 3).all(axis=1)]
sst_season_day_adjusted=sst_summer_day_adjusted.append(sst_autumn_day_adjusted.append(sst_winter_day_adjusted.append(sst_spring_day_adjusted)))

#### Weeks

##### Remove Outliers

In [ ]:
sst_weeks_day_removed_outliers={}
for i in range(1, 53):
    sst_weeks_day_removed_outliers[i]=original_sst_weeks_day[i][original_sst_weeks_day[i].apply(lambda x: np.abs(x - x.mean()) / x.std() < 3).all(axis=1)]

##### Calculate Means

In [ ]:
sst_weeks_day_mean={}
for i in range(1, 53):
    sst_weeks_day_mean[i]=sst_weeks_day_removed_outliers[i].mean()

##### Create Adjusted Dataframes

In [ ]:
sst_weeks_day_adjusted=original_wavedata.drop(["Hs", "Hmax", "Tz", "Tp", "Dir_Tp TRUE"], axis=1).between_time('6:00', '18:00')
for i in range(1, 53):
    sst_weeks_day_adjusted[sst_weeks_day_adjusted["Date/Time"].dt.week==i]=sst_weeks_day_adjusted[sst_weeks_day_adjusted["Date/Time"].dt.week==i].replace(np.nan, sst_weeks_day_mean[i])

##### Remove Outliers from Adjusted Dataframes

In [ ]:
sst_weeks_day_adjusted=sst_weeks_day_adjusted.drop(["Date/Time"], axis=1)
sst_weeks_day_adjusted=sst_weeks_day_adjusted[sst_weeks_day_adjusted.apply(lambda x: np.abs(x - x.mean()) / x.std() < 3).all(axis=1)]

### Night

#### Months

##### Remove Outliers

In [ ]:
sst_months_night_removed_outliers={}
for i in range(1, 13):
    sst_months_night_removed_outliers[i]=original_sst_months_night[i][original_sst_months_night[i].apply(lambda x: np.abs(x - x.mean()) / x.std() < 3).all(axis=1)]

##### Calculate Means

In [ ]:
sst_months_night_mean={}
for i in range(1, 13):
    sst_months_night_mean[i]=sst_months_night_removed_outliers[i].mean()

##### Create Adjusted Dataframes

In [ ]:
sst_months_night_adjusted=original_wavedata.drop(["Hs", "Hmax", "Tz", "Tp", "Dir_Tp TRUE"], axis=1).between_time('18:00', '6:00')
for i in range(1, 13):
    sst_months_night_adjusted[sst_months_night_adjusted["Date/Time"].dt.month==i]=sst_months_night_adjusted[sst_months_night_adjusted["Date/Time"].dt.month==i].replace(np.nan, sst_months_night_mean[i])

##### Remove Outliers from Adjusted Dataframes

In [ ]:
sst_months_night_adjusted=sst_months_night_adjusted.drop(["Date/Time"], axis=1)
sst_months_night_adjusted=sst_months_night_adjusted[sst_months_night_adjusted.apply(lambda x: np.abs(x - x.mean()) / x.std() < 3).all(axis=1)]

#### Seasons

##### Remove Outliers

In [ ]:
sst_summer_night_removed_outliers=original_sst_summer_night[original_sst_summer_night.apply(lambda x: np.abs(x - x.mean()) / x.std() < 3).all(axis=1)]
sst_autumn_night_removed_outliers=original_sst_autumn_night[original_sst_autumn_night.apply(lambda x: np.abs(x - x.mean()) / x.std() < 3).all(axis=1)]
sst_winter_night_removed_outliers=original_sst_winter_night[original_sst_winter_night.apply(lambda x: np.abs(x - x.mean()) / x.std() < 3).all(axis=1)]
sst_spring_night_removed_outliers=original_sst_spring_night[original_sst_spring_night.apply(lambda x: np.abs(x - x.mean()) / x.std() < 3).all(axis=1)]

##### Calculate Means

In [ ]:
sst_summer_night_mean=sst_summer_night_removed_outliers.mean()
sst_autumn_night_mean=sst_autumn_night_removed_outliers.mean()
sst_winter_night_mean=sst_winter_night_removed_outliers.mean()
sst_spring_night_mean=sst_spring_night_removed_outliers.mean()

##### Create Adjusted Dataframes

In [ ]:
sst_summer_night_adjusted=original_sst_summer_night
sst_autumn_night_adjusted=original_sst_autumn_night
sst_winter_night_adjusted=original_sst_winter_night
sst_spring_night_adjusted=original_sst_spring_night
sst_summer_night_adjusted=sst_summer_night_adjusted.replace(np.nan, sst_summer_night_mean)
sst_autumn_night_adjusted=sst_autumn_night_adjusted.replace(np.nan, sst_autumn_night_mean)
sst_winter_night_adjusted=sst_winter_night_adjusted.replace(np.nan, sst_winter_night_mean)
sst_spring_night_adjusted=sst_spring_night_adjusted.replace(np.nan, sst_spring_night_mean)

##### Remove Outliers from Adjusted Dataframes

In [ ]:
sst_summer_night_adjusted=sst_summer_night_adjusted[sst_summer_night_adjusted.apply(lambda x: np.abs(x - x.mean()) / x.std() < 3).all(axis=1)]
sst_autumn_night_adjusted=sst_autumn_night_adjusted[sst_autumn_night_adjusted.apply(lambda x: np.abs(x - x.mean()) / x.std() < 3).all(axis=1)]
sst_winter_night_adjusted=sst_winter_night_adjusted[sst_winter_night_adjusted.apply(lambda x: np.abs(x - x.mean()) / x.std() < 3).all(axis=1)]
sst_spring_night_adjusted=sst_spring_night_adjusted[sst_spring_night_adjusted.apply(lambda x: np.abs(x - x.mean()) / x.std() < 3).all(axis=1)]
sst_season_night_adjusted=sst_summer_night_adjusted.append(sst_autumn_night_adjusted.append(sst_winter_night_adjusted.append(sst_spring_night_adjusted)))

#### Weeks

##### Remove Outliers

In [ ]:
sst_weeks_night_removed_outliers={}
for i in range(1, 53):
    sst_weeks_night_removed_outliers[i]=original_sst_weeks_night[i][original_sst_weeks_night[i].apply(lambda x: np.abs(x - x.mean()) / x.std() < 3).all(axis=1)]

##### Calculate Means

In [ ]:
sst_weeks_night_mean={}
for i in range(1, 53):
    sst_weeks_night_mean[i]=sst_weeks_night_removed_outliers[i].mean()

##### Create Adjusted Dataframes

In [ ]:
sst_weeks_night_adjusted=original_wavedata.drop(["Hs", "Hmax", "Tz", "Tp", "Dir_Tp TRUE"], axis=1).between_time('18:00', '6:00')
for i in range(1, 53):
    sst_weeks_night_adjusted[sst_weeks_night_adjusted["Date/Time"].dt.week==i]=sst_weeks_night_adjusted[sst_weeks_night_adjusted["Date/Time"].dt.week==i].replace(np.nan, sst_weeks_night_mean[i])

##### Remove Outliers from Adjusted Dataframes

In [ ]:
sst_weeks_night_adjusted=sst_weeks_night_adjusted.drop(["Date/Time"], axis=1)
sst_weeks_night_adjusted=sst_weeks_night_adjusted[sst_weeks_night_adjusted.apply(lambda x: np.abs(x - x.mean()) / x.std() < 3).all(axis=1)]

### Compile Adjusted Dataframes

In [ ]:
sst_months_adjusted=sst_months_day_adjusted.append(sst_months_night_adjusted)
sst_season_adjusted=sst_season_day_adjusted.append(sst_season_night_adjusted)
sst_weeks_adjusted=sst_weeks_day_adjusted.append(sst_weeks_night_adjusted)

# Create Completed Dataframes

In [ ]:
wavedata_dir_tp_months_sst_months=pd.merge(original_wavedata.drop(['Date/Time', 'Dir_Tp TRUE', 'SST'], axis=1), dir_tp_months_adjusted.join(sst_months_adjusted), 'right', right_index=True, left_on='Date/Time')
wavedata_dir_tp_months_sst_season=pd.merge(original_wavedata.drop(['Date/Time', 'Dir_Tp TRUE', 'SST'], axis=1), dir_tp_months_adjusted.join(sst_season_adjusted), 'right', right_index=True, left_on='Date/Time')
wavedata_dir_tp_months_sst_weeks=pd.merge(original_wavedata.drop(['Date/Time', 'Dir_Tp TRUE', 'SST'], axis=1), dir_tp_months_adjusted.join(sst_weeks_adjusted), 'right', right_index=True, left_on='Date/Time')
wavedata_dir_tp_season_sst_months=pd.merge(original_wavedata.drop(['Date/Time', 'Dir_Tp TRUE', 'SST'], axis=1), dir_tp_season_adjusted.join(sst_months_adjusted), 'right', right_index=True, left_on='Date/Time')
wavedata_dir_tp_season_sst_season=pd.merge(original_wavedata.drop(['Date/Time', 'Dir_Tp TRUE', 'SST'], axis=1), dir_tp_season_adjusted.join(sst_season_adjusted), 'right', right_index=True, left_on='Date/Time')
wavedata_dir_tp_season_sst_weeks=pd.merge(original_wavedata.drop(['Date/Time', 'Dir_Tp TRUE', 'SST'], axis=1), dir_tp_season_adjusted.join(sst_weeks_adjusted), 'right', right_index=True, left_on='Date/Time')
wavedata_dir_tp_weeks_sst_months=pd.merge(original_wavedata.drop(['Date/Time', 'Dir_Tp TRUE', 'SST'], axis=1), dir_tp_weeks_adjusted.join(sst_months_adjusted), 'right', right_index=True, left_on='Date/Time')
wavedata_dir_tp_weeks_sst_season=pd.merge(original_wavedata.drop(['Date/Time', 'Dir_Tp TRUE', 'SST'], axis=1), dir_tp_weeks_adjusted.join(sst_season_adjusted), 'right', right_index=True, left_on='Date/Time')
wavedata_dir_tp_weeks_sst_weeks=pd.merge(original_wavedata.drop(['Date/Time', 'Dir_Tp TRUE', 'SST'], axis=1), dir_tp_weeks_adjusted.join(sst_weeks_adjusted), 'right', right_index=True, left_on='Date/Time')

# Create Normalised Dataframes

In [ ]:
normalised_wavedata_dir_tp_months_sst_months=normalise_dataframe(wavedata_dir_tp_months_sst_months).sort_index()
normalised_wavedata_dir_tp_months_sst_season=normalise_dataframe(wavedata_dir_tp_months_sst_season).sort_index()
normalised_wavedata_dir_tp_months_sst_weeks=normalise_dataframe(wavedata_dir_tp_months_sst_weeks).sort_index()
normalised_wavedata_dir_tp_season_sst_months=normalise_dataframe(wavedata_dir_tp_season_sst_months).sort_index()
normalised_wavedata_dir_tp_season_sst_season=normalise_dataframe(wavedata_dir_tp_season_sst_season).sort_index()
normalised_wavedata_dir_tp_season_sst_weeks=normalise_dataframe(wavedata_dir_tp_season_sst_weeks).sort_index()
normalised_wavedata_dir_tp_weeks_sst_months=normalise_dataframe(wavedata_dir_tp_weeks_sst_months).sort_index()
normalised_wavedata_dir_tp_weeks_sst_season=normalise_dataframe(wavedata_dir_tp_weeks_sst_season).sort_index()
normalised_wavedata_dir_tp_weeks_sst_weeks=normalise_dataframe(wavedata_dir_tp_weeks_sst_weeks).sort_index()